In [ ]:
import pickle
import os
from tqdm import tqdm
import numpy as np

directory = 'data/train'

length_list = []
valence_values=[]

recordings = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith('.pkl'):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
            if data['valence'] != 2.333:
                length_list.append(len(data['audio_data']))
                valence_values.append(data['valence'])
                recordings.append(data['audio_data'])

len(recordings)

In [ ]:

max_length = max(len(array) for array in recordings)  # Find the maximum length

# Pad each array to have the maximum length
padded_arrays = np.array([np.pad(array, (0, max_length - len(array)), mode='constant') for array in recordings])

In [ ]:
valence_values = np.array(valence_values)
valence_values

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data and labels into training and testing sets
X_train, X_test_help, y_train, y_test_help = train_test_split(padded_arrays, valence_values, test_size=0.4, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_test_help, y_test_help, test_size=0.5, random_state=42)


# Length of recordings distribution

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.hist(length_list, bins=100)
plt.show()

print(len(length_list))
print(len(np.unique(np.array(length_list))))



# Valence distribution

In [ ]:
valence_dict = {}

for i in valence_values:
    if i not in valence_dict:
        valence_dict[i] = 1
    else:
        valence_dict[i] += 1
valence_dict


In [ ]:
import torch

# Convert input data
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)  # Use float32 for input features

# Convert labels
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Use long if your labels are for classification tasks

from torch.utils.data import TensorDataset, DataLoader

# Create a dataset from tensors
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

# Create a DataLoader
batch_size = 64  # You can adjust the batch size depending on your system's capability
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)



X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)




In [ ]:
y_test_tensor

# Plan


### 1. Neural network with padding

### 2. Neural network with 1 input 1 hidden 1 output layer

### 3. Optimizer : ADAM 

### 4. Loss function - Categorical cross entropy



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, activation_function):
        """
        Initialize the MLP model.

        Parameters:
        - input_size (int): Size of the input features.
        - hidden_sizes (list): List containing the sizes of hidden layers.
        - output_size (int): Size of the output layer.
        - activation_function (torch.nn.Module): Activation function for hidden layers.
        """
        super(MLP, self).__init__()

        # Set random seed for reproducibility
        torch.manual_seed(42)

        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size

        # Create hidden layers and activations dynamically
        self.layers = nn.ModuleList()

        for i in range(len(hidden_sizes)):
            # Linear layer
            self.layers.append(nn.Linear(input_size if i == 0 else hidden_sizes[i - 1], hidden_sizes[i]))

            # Activation function (except for the last layer)
            self.layers.append(activation_function())

        # Append the ouptu layer
        self.layers.append(nn.Softmax(dim=1))
        

    def forward(self, x):
 
        # Flatten the input
        x = x.view(-1, self.input_size)

        # Forward pass through hidden layers with activation functions
        for layer in self.layers:
            x = layer(x)

        return x


In [ ]:
# Training Cycle

def train_model(MLP_model, optimizer, num_epochs):
    # Define the loss function
    criterion = nn.CrossEntropyLoss()
    # Training loop
    for epoch in range(num_epochs):
        total_loss = 0

        for inputs, labels in test_loader:
            inputs = inputs.view(-1, max_length)  # Flatten the images
            outputs = MLP_model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Compute the loss
            loss.backward()  # Backward pass

            # Update weights using the step function of our custom ADAM optimizer
            optimizer.step()

            # Store the loss. loss.item() gets the value in a tensor. This only works for scalars.
            total_loss += loss.item()
    
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}')
    

In [ ]:
def evaluate_model(MLP_model, test_loader):
    # Model Evaluation
    predicted_labels = []
    true_labels = []
    with torch.no_grad():
        MLP_model.eval()  # Set the model to evaluation mode
        total_correct = 0
        total_samples = 0

        for inputs, labels in test_loader:
            inputs = inputs.view(-1, max_length)  # Flatten the test images
            outputs = MLP_model(inputs)
            _, predicted = torch.max(outputs, 1)
            predicted_labels.extend(predicted.numpy())
            true_labels.extend(labels.numpy())
            
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
    

    accuracy = total_correct / total_samples
    print(f'Test Accuracy: {accuracy * 100:.2f}%')
    print(predicted_labels)
    print("True Labels:")
    print(true_labels)


In [ ]:

# Initialize the model
input_size = max_length  # MNIST images are 28x28 pixels
hidden_size = [16, 16]
output_size = 17  # 10 classes for digits 0 to 9
activation_function = nn.ReLU
num_epochs = 20

# Create the model
model = MLP(input_size, hidden_size, output_size, activation_function)

# Here we create an instance of our custom_SGD class, giving as arguments the learning rate.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_model(model, optimizer, num_epochs=num_epochs)

# Evaluate the model
evaluate_model(model, test_loader)
